In [1]:
# !pip install h2o

In [2]:
import pandas as pd
import json
import os
import re

from datetime import datetime

import h2o
from h2o.automl import H2OAutoML

In [3]:
j = open('param_h2o.json', 'r').read()
param = json.loads(j)
param

{'sourcePath': 'https://s3model.blob.core.windows.net/prodata/merged_proc_data.csv',
 'savingPath': 'https://s3model.blob.core.windows.net/modeldata/',
 'target': 'average_ticket',
 'excludeAlgos': 'DeepLearning',
 'algos': ['DRF', 'GLM', 'XGBoost', 'GBM', 'DeepLearning', 'StackedEnsemble'],
 'maxModels': '10',
 'seed': '13',
 'maxTrainingTime': '30',
 'classification': 'False'}

In [4]:
source_path = param['sourcePath']
saving_path = param['savingPath']
target = param['target']
exclude_algos = [param['excludeAlgos']]
max_models = int(param['maxModels'])
seed = int(param['seed'])
max_training_time = int(param['maxTrainingTime'])
classification = param['classification']

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 15 hours 30 mins
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_hobbes_pu0o06
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,993 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [6]:
df = h2o.import_file(path=source_path)
train, test = df.split_frame(ratios=[0.8], seed = seed)

aml = H2OAutoML(
    max_models = max_models, 
    max_runtime_secs = max_training_time, 
    exclude_algos = exclude_algos, 
    balance_classes=True, 
    seed = seed
)

aml.train(training_frame = train, y = target)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
02:46:06.928: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

███████████████████
02:46:17.932: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_17_20220121_24606


Model Summary: 


,,number_of_trees
0,,8.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 31.522505031118826
RMSE: 5.614490629711552
MAE: 4.270427672718242
RMSLE: 0.09696553317860397
Mean Residual Deviance: 31.522505031118826

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 35.39367254203476
RMSE: 5.949258150562535
MAE: 4.303081102087595
RMSLE: 0.10272355891635357
Mean Residual Deviance: 35.39367254203476

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,4.303081,0.520164,4.554263,4.535135,4.562425,4.489631,3.373952
1,mean_residual_deviance,35.393673,5.225140,38.118850,37.339504,38.656326,36.712627,26.141054
2,mse,35.393673,5.225140,38.118850,37.339504,38.656326,36.712627,26.141054
3,r2,0.899935,0.015882,0.892100,0.892529,0.889789,0.897339,0.927917
4,residual_deviance,35.393673,5.225140,38.118850,37.339504,38.656326,36.712627,26.141054
5,rmse,5.934800,0.463439,6.174047,6.110606,6.217421,6.059094,5.112832
6,rmsle,0.102525,0.007136,0.106399,0.105730,0.106640,0.103958,0.089898



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-01-21 03:46:14,7.840 sec,0.0,58.494167,55.382435,3421.567534
1,,2022-01-21 03:46:16,9.613 sec,5.0,11.397502,9.866478,129.903062
2,,2022-01-21 03:46:17,10.266 sec,8.0,5.614491,4.270428,31.522505



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,price_range,7.460582e+06,1.000000,0.636962
1,order_total_amount,1.476298e+06,0.197880,0.126042
2,minimum_order_value,8.972041e+05,0.120259,0.076601
3,delivery_time,2.769564e+05,0.037123,0.023646
4,delivery_address_longitude,2.668274e+05,0.035765,0.022781
5,merchant_zip_code,2.057129e+05,0.027573,0.017563
6,enabled.True,1.909349e+05,0.025592,0.016301
7,takeout_time,1.157688e+05,0.015517,0.009884
8,res_created_at,8.285427e+04,0.011106,0.007074
9,delivery_address_zip_code,8.012322e+04,0.010740,0.006841



See the whole table with table.as_data_frame()


In [7]:
best_model = aml.get_best_model()

model_info = best_model.model_performance(test)

In [8]:
local_path = os.getcwd()+'/'

In [9]:
model_path_1 = h2o.save_model(model=best_model,path=local_path, force=True)

In [10]:
best_model = model_path_1.split('/')[int(len(model_path_1.split('/'))-1)]

model_id = best_model.split('_')[-2]+best_model.split('_')[-1]
model_name = best_model.split('_')[0]+'_'+model_id
model_date = best_model.split('_')[-2]

In [11]:
test_name = 'test_' + model_id
train_name = 'train_' + model_id

In [12]:
test.as_data_frame().convert_dtypes().to_pickle(test_name + '.csv')
train.as_data_frame().convert_dtypes().to_pickle(train_name + '.csv')

In [13]:
model_path = saving_path + model_name

try:
    aic = model_info.aic()
except:
    aic = str(0)
    
info1 = {
        'id': str(model_id),
        'name': str(model_name),
        'date': datetime. strptime(model_date, '%Y%m%d'),
        "h2o_version": str(h2o.__version__),
        "seed": str(seed),
        "target": str(target),
        "train_path": str(saving_path + test_name+ '.csv'),
        "test_path": str(saving_path + train_name+ '.csv'),
        'model_path':str(saving_path + model_name),
        'max_runtime': str(aml.max_runtime_secs),
        'classification':str(classification),
        'columns':str(list(aml.varimp().index)),
        'varimp':str(aml.varimp().to_json())
}


if classification == 'False':
    info2 = {
        'metrics':{
            'mse':str(model_info.mse()),
            'rmse':str(model_info.rmse()),
            'mae':str(model_info.mae()),
            'rmsle':str(model_info.rmsle()),
            'r2':str(model_info.r2()),
            'mean_resid_deviance':str(model_info.mean_residual_deviance()),
            'null_degrees_of_freedom':str(model_info.null_degrees_of_freedom()),
            'resid_degrees_of_freedom':str(model_info.residual_degrees_of_freedom()),
            'null_deviance':str(model_info.null_deviance()),
            'residual_deviance':str(model_info.residual_deviance()),
            'aic':aic,
        }
    }
    
info = info1.copy()
info.update(info2)

In [59]:
info_new = pd.DataFrame([info])

info_path = local_path + 'info.csv'

try:
    info_old = pd.read_csv(info_path)
    if info_old.columns[0] == 'Unnamed: 0':
        info_old = info_old.drop(info_old.columns[0], axis=1)
    
    info_save = pd.concat([info_old, info_new])
    info_save.to_csv(info_path)
except:
    info_new.to_csv(info_path)

In [60]:
# os.rename(model_path_1, local_path + model_name)

In [61]:
# info_new.to_csv(info_path)

In [62]:
df = pd.read_csv('info.csv')

In [63]:
df

,Unnamed: 0,name,date,h2o_version,seed,target,train_path,test_path,model_path,max_runtime,classification,columns,varimp,metrics,id
0,0,XGBoost_2022012194719,2022-01-21,3.34.0.7,3,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012194719...,https://s3model.blob.core.windows.net/modeldata/train_202201219471...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012194719,30,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.00002668,""c...","{'mse': '26.314887710146255', 'rmse': '5.129803866635279', 'mae': ...",NaN
1,1,XGBoost_2022012195029,2022-01-21 00:00:00,3.34.0.7,2,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012195029...,https://s3model.blob.core.windows.net/modeldata/train_202201219502...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012195029,45,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.0001232925,...","{'mse': '18.789868930261495', 'rmse': '4.334728241800343', 'mae': ...",2.022012e+12
2,2,XGBoost_2022012195336,2022-01-21 00:00:00,3.34.0.7,2,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012195336...,https://s3model.blob.core.windows.net/modeldata/train_202201219533...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012195336,45,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.0001232925,...","{'mse': '18.789868930261495', 'rmse': '4.334728241800343', 'mae': ...",2.022012e+12
3,3,XGBoost_2022012124606,2022-01-21 00:00:00,3.34.0.7,13,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012124606...,https://s3model.blob.core.windows.net/modeldata/train_202201212460...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012124606,30,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.0002162418,...","{'mse': '36.38087374803645', 'rmse': '6.031655970629994', 'mae': '...",2.022012e+12
4,4,XGBoost_2022012124606,2022-01-21 00:00:00,3.34.0.7,13,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012124606...,https://s3model.blob.core.windows.net/modeldata/train_202201212460...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012124606,30,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.0002162418,...","{'mse': '36.38087374803645', 'rmse': '6.031655970629994', 'mae': '...",2.022012e+12
5,0,XGBoost_2022012124606,2022-01-21 00:00:00,3.34.0.7,13,average_ticket,https://s3model.blob.core.windows.net/modeldata/test_2022012124606...,https://s3model.blob.core.windows.net/modeldata/train_202201212460...,https://s3model.blob.core.windows.net/modeldata/XGBoost_2022012124606,30,False,"['order_created_at', 'cs_created_at', 'customer_name', 'delivery_a...","{""XGBoost_1"":{""order_created_at"":0.0,""cs_created_at"":0.0002162418,...","{'mse': '36.38087374803645', 'rmse': '6.031655970629994', 'mae': '...",2.022012e+12


In [31]:
if df.columns[0] == 'Unnamed: 0':
    df = df.drop(df.columns[0], axis=1)

In [32]:
df.to_csv('info.csv')

In [33]:
df2 = pd.read_csv('info.csv')